В этом соревновании вам нужно предсказать популярность объявления о продаже домов в Нью-Йорке, основываясь на текстовом описании, локации, количестве спален, цене etc.

Вы решаете задачу классификации: нужно предсказать TARGET − low, medium или high. Метрка − точность.

train.csv − датасет для обучения (38 тысяч примеров, 12 фичей + лейбл)
test.csv − датасет для оценивания решений (11 тысяч примеров, 12 фичей)
Фичи:

bathrooms: количество ванных
bedrooms: количество спален
building_id: ID здания
created: дата создания объявления
display_address: адрес (текстом)
features: список разных полезных текстовых тегов
latitude: ширина (координата)
longitude: долгода (координата)
listing_id: ID объявления
manager_id: ID риелтора
photos: количество фотографий у объявления
price: цена в долларах
TARGET: целевая переменная; "low", "medium" или "high"

In [210]:
import numpy as np
import pandas as pd

In [211]:
X = pd.read_csv('data/train.csv')
X

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,['No pets'],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low
34542,50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium
34543,111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low
34544,71184,1.0,2,8754cae39f6e053974aa2337017eb3c1,2016-05-14 02:27:47,CooperCooper.com :: Listing ID #10_0385; 400 W...,400 West 56th Street,"['Laundry in Unit', 'Dishwasher', 'Cats Allowe...",40.7673,7010187,-73.9866,6e5c10246156ae5bdcd9b487ca99d96a,['https://photos.renthop.com/2/7010187_ab1db5d...,3400,400 West 56th Street,medium


In [212]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34546 entries, 0 to 34545
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               34546 non-null  int64  
 1   bathrooms        34546 non-null  float64
 2   bedrooms         34546 non-null  int64  
 3   building_id      34546 non-null  object 
 4   created          34546 non-null  object 
 5   description      33509 non-null  object 
 6   display_address  34458 non-null  object 
 7   features         34546 non-null  object 
 8   latitude         34546 non-null  float64
 9   listing_id       34546 non-null  int64  
 10  longitude        34546 non-null  float64
 11  manager_id       34546 non-null  object 
 12  photos           34546 non-null  object 
 13  price            34546 non-null  int64  
 14  street_address   34542 non-null  object 
 15  TARGET           34546 non-null  object 
dtypes: float64(3), int64(4), object(9)
memory usage: 4.2+ MB


In [213]:
X.isna().sum()

Id                    0
bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
TARGET                0
dtype: int64

Так как нельзя использовать обработку текстов, то можно удалить столбец description, display_address, features, а
 listing_id, building_id - просто бесполезен

In [214]:
print(len(set(X['manager_id'])), len(X['manager_id']))
print(len(set(X['building_id'])), len(X['building_id']))
print(len(set(X['listing_id'])), len(X['listing_id']))

3134 34546
6378 34546
34546 34546


In [215]:
from collections import Counter

manager_high = Counter(X[X['TARGET'] == 'high']['manager_id'])
manager_medium = Counter(X[X['TARGET'] == 'medium']['manager_id'])
manager_low = Counter(X[X['TARGET'] == 'low']['manager_id'])

In [216]:
manager_high

Counter({'12c0a30e296faa0dfd422fe918d7d4f4': 31,
         '79c834b88dc49e64b6cf46a7de8dc5a2': 1,
         '603575a3da76fa757cd6451f3707819d': 12,
         'd4e72926d837c6ca5849c62cead367b4': 10,
         'b944623e2af9b605eb0cba5236ee5f8e': 4,
         'c44ea3e83ff2048561ab5c21a2d0c90e': 4,
         'e6472c7237327dd3903b3d6f6a94515a': 133,
         'eaa703a11f54569e6a4c9b94fbf7605f': 1,
         '1f83758a257b3ded8e8cce14b14bf533': 2,
         '13018d720355f7be3325c544876e2c7a': 1,
         '1badf2eefe8228387979149413ec071b': 2,
         '9df32cb8dda19d3222d66e69e258616b': 12,
         '4c4362be5604f9054c2d6d4f82f9d24e': 2,
         'a7b1843113e8780eb0f881ab1596a9a8': 2,
         '24e97efd60ebc044c1817e6c12598527': 16,
         'b2f322e6ea73856e1ee88bf0d08eb54c': 5,
         'e90f2ded843cdb2efd65ef47d9fc8029': 23,
         '18464b5400971c49bfd12a59c5477174': 11,
         'be1a7235af23dd77d0d1029dd5bfa5e9': 35,
         'ea370ccce5abc74f0c46e6b684d24c91': 2,
         'be8b38e3002bc1f26202

In [217]:
manager_medium

Counter({'4bdc3d8c1aaa90d997ce2cb77680679b': 28,
         'e5808a5e6cc13988fe596704428d38d5': 19,
         '0532e99449bc0266d211c5ab9e820f48': 7,
         '6e5c10246156ae5bdcd9b487ca99d96a': 40,
         '82c4fff4064b74ef4333a17a20f0d899': 3,
         '9cbfa7a31ae9dfba70b4383636d99788': 24,
         '3b630ec9cb6eee53b92cfac7f42e3bf4': 7,
         'd2bce61e0e0079ebdc8c281e415e045b': 35,
         'e6472c7237327dd3903b3d6f6a94515a': 429,
         'eb5a6c4c2eb6f3e2a085c477afd5a815': 59,
         '7d27250d99040a907ac66d4c8f62ac58': 2,
         '2983e45f7e0ad87d677dacd13e362785': 16,
         'dbbb6b990661b1e507a387f019bcb1a0': 65,
         '69f035007d270281a624ab2cc1d2aba0': 15,
         'bc9a2ff7c931ce7de08f50d429134a16': 13,
         '02e17b21a1814fb10336b2ee8ceb3e79': 51,
         '92ef9986d12ec39a35fcb572772e948a': 25,
         '65322e6138b5aaa5467cdc31a19112f9': 5,
         'b28bbb4fc65aab175a3aeba83684843d': 1,
         'd1737922fe92ccb0dc37ba85589e6415': 16,
         'd1710a0b5ec069f

In [218]:
manager_low

Counter({'d69d4e111612dd12ef864031c1148543': 9,
         'e6472c7237327dd3903b3d6f6a94515a': 1258,
         '6fba9b3a8327c607b8b043716efee684': 28,
         '70c9035cc23844487e1754b0d58577f5': 15,
         'd2bce61e0e0079ebdc8c281e415e045b': 154,
         '3734dac511776b0990e90a94d71c8251': 17,
         '8262449f40e9117f7a9ea49b4a333993': 137,
         'c29f49252742cb38e9959bad292ee7e7': 5,
         'b05bf5018f5763c9befff759e35cc8bd': 10,
         'acddfbbc232e43b8100e3afc34f26833': 21,
         '07c511151921e268b260a2bc96399704': 9,
         '6e5c10246156ae5bdcd9b487ca99d96a': 469,
         '8a507da1e21e16db582b77c19b2fdea0': 21,
         '8f5a9c893f6d602f4953fcc0b8e6e9b4': 277,
         '75f38d077366d7964c2f3bb80c3e7b1d': 39,
         '536aaedf27d13fb487c142dae8133211': 37,
         'f5b4f30a5e431fcea734cea1d5e740bb': 12,
         'cb87dadbca78fad02b388dc9e8f25a5b': 87,
         'cf36aeba79d8e830f8353e30ab551520': 19,
         'a7b1843113e8780eb0f881ab1596a9a8': 11,
         '605757e

Заменим manager_id на 4 новых столбца(имеет ли менеджер > 25 объявлений с определенным таргетом)

In [219]:
def popular_manager_high(manager):
    if manager_high[manager] > 25:
        return 1
    return 0


def popular_manager_medium(manager):
    if manager_medium[manager] > 25:
        return 1
    return 0


def popular_manager_low(manager):
    if manager_low[manager] > 25:
        return 1
    return 0


def manager_not_popular(manager):
    if manager_high[manager] < 25 and manager_medium[manager] < 25 and manager_low[manager] < 25:
        return 1
    return 0


def manager_only_high(manager):
    if manager in manager_high and manager not in manager_low and manager not in manager_medium:
        return 1
    return 0


def manager_only_medium(manager):
    if manager not in manager_high and manager not in manager_low and manager in manager_medium:
        return 1
    return 0


def manager_only_low(manager):
    if manager not in manager_high and manager in manager_low and manager not in manager_medium:
        return 1
    return 0


def manager_low_medium(manager):
    if manager not in manager_high and manager not in manager_low and manager in manager_medium:
        return 1
    return 0


def manager_low_high(manager):
    if manager in manager_high and manager in manager_low and manager not in manager_medium:
        return 1
    return 0


def manager_medium_high(manager):
    if manager in manager_high and manager not in manager_low and manager in manager_medium:
        return 1
    return 0


def manager_low_medium_high(manager):
    if manager in manager_high and manager in manager_low and manager in manager_medium:
        return 1
    return 0

In [220]:
X['popular_manager_low'] = X['manager_id'].apply(popular_manager_low)
X['popular_manager_medium'] = X['manager_id'].apply(popular_manager_medium)
X['popular_manager_high'] = X['manager_id'].apply(popular_manager_high)
X['manager_not_popular'] = X['manager_id'].apply(manager_not_popular)
X['manager_high'] = X['manager_id'].apply(manager_only_high)
X['manager_medium'] = X['manager_id'].apply(manager_only_medium)
X['manager_low'] = X['manager_id'].apply(manager_only_low)
X['manager_low_medium'] = X['manager_id'].apply(manager_low_medium)
X['manager_low_high'] = X['manager_id'].apply(manager_low_high)
X['manager_medium_high'] = X['manager_id'].apply(manager_medium_high)
X['manager_low_medium_high'] = X['manager_id'].apply(manager_low_medium_high)

Разобьем по временам года

In [221]:
X['month'] = pd.DatetimeIndex(X['created']).month
# там только 4,5,6 месяц в итоге (быстро посмотрел вручную)

In [222]:
X['April'] = X['month'].apply(lambda x: 1 if x == 4 else 0)
X['June'] = X['month'].apply(lambda x: 1 if x == 6 else 0)
X['May'] = X['month'].apply(lambda x: 1 if x == 5 else 0)

photos сделаем просто количеством

In [223]:
X['photos'] = X['photos'].apply(lambda x: len(x))

Удалим все ненужное

In [224]:
X = X.drop(columns=['description', 'display_address', 'features', 'listing_id', 'building_id', 'month', 'Id', 'created',
                    'manager_id', 'street_address'])


In [225]:
X

,bathrooms,bedrooms,latitude,longitude,photos,price,TARGET,popular_manager_low,popular_manager_medium,popular_manager_high,...,manager_high,manager_medium,manager_low,manager_low_medium,manager_low_high,manager_medium_high,manager_low_medium_high,April,June,May
0,1.0,3,40.7647,-73.9918,231,4495,medium,1,1,0,...,0,0,0,0,0,0,1,0,0,1
1,1.0,1,40.7275,-74.0322,1001,2570,medium,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1.0,0,40.7798,-73.9751,462,1795,low,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1.0,2,40.7081,-74.0065,308,3400,low,1,1,1,...,0,0,0,0,0,0,1,0,0,1
4,1.0,3,40.7416,-74.0025,231,5695,low,1,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,40.7653,-73.9248,308,1950,low,1,1,1,...,0,0,0,0,0,0,1,0,1,0
34542,1.0,0,40.7239,-73.9901,462,2600,medium,1,1,0,...,0,0,0,0,0,0,1,0,0,1
34543,2.0,2,40.7437,-73.9585,1155,5110,low,1,1,1,...,0,0,0,0,0,0,1,1,0,0
34544,1.0,2,40.7673,-73.9866,231,3400,medium,1,1,0,...,0,0,0,0,0,0,1,0,0,1


Найти район каждой квартиры по координатам занимает очень много времени, поэтому найдем расстояние до центра

In [226]:
from math import sin, cos, sqrt, atan2, radians

R = 6373.0
NY_latitude = 40.730610
NY_longitude = -73.935242


def get_distance(latitude, longitude):
    result = []
    for i in range(len(latitude)):
        dlat = latitude[i] - NY_latitude
        dlon = longitude[i] - NY_longitude
        a = sin(dlat / 2) ** 2 + cos(NY_latitude) * cos(latitude[i]) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        result.append(R * c)
    return result


X['distance'] = get_distance(X['latitude'], X['longitude'])
X['distance'] = X['distance'].apply(lambda x: int(x))
X

,bathrooms,bedrooms,latitude,longitude,photos,price,TARGET,popular_manager_low,popular_manager_medium,popular_manager_high,...,manager_medium,manager_low,manager_low_medium,manager_low_high,manager_medium_high,manager_low_medium_high,April,June,May,distance
0,1.0,3,40.7647,-73.9918,231,4495,medium,1,1,0,...,0,0,0,0,0,1,0,0,1,419
1,1.0,1,40.7275,-74.0322,1001,2570,medium,0,0,0,...,0,0,0,0,0,1,0,1,0,614
2,1.0,0,40.7798,-73.9751,462,1795,low,0,0,0,...,0,0,0,0,0,1,0,0,1,402
3,1.0,2,40.7081,-74.0065,308,3400,low,1,1,1,...,0,0,0,0,0,1,0,0,1,473
4,1.0,3,40.7416,-74.0025,231,5695,low,1,0,0,...,0,0,0,0,0,1,0,1,0,432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,40.7653,-73.9248,308,1950,low,1,1,1,...,0,0,0,0,0,1,0,1,0,230
34542,1.0,0,40.7239,-73.9901,462,2600,medium,1,1,0,...,0,0,0,0,0,1,0,0,1,349
34543,2.0,2,40.7437,-73.9585,1155,5110,low,1,1,1,...,0,0,0,0,0,1,1,0,0,169
34544,1.0,2,40.7673,-73.9866,231,3400,medium,1,1,0,...,0,0,0,0,0,1,0,0,1,401


In [227]:
X = X.drop(columns=['latitude', 'longitude'])
X

,bathrooms,bedrooms,photos,price,TARGET,popular_manager_low,popular_manager_medium,popular_manager_high,manager_not_popular,manager_high,manager_medium,manager_low,manager_low_medium,manager_low_high,manager_medium_high,manager_low_medium_high,April,June,May,distance
0,1.0,3,231,4495,medium,1,1,0,0,0,0,0,0,0,0,1,0,0,1,419
1,1.0,1,1001,2570,medium,0,0,0,1,0,0,0,0,0,0,1,0,1,0,614
2,1.0,0,462,1795,low,0,0,0,1,0,0,0,0,0,0,1,0,0,1,402
3,1.0,2,308,3400,low,1,1,1,0,0,0,0,0,0,0,1,0,0,1,473
4,1.0,3,231,5695,low,1,0,0,0,0,0,0,0,0,0,1,0,1,0,432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,308,1950,low,1,1,1,0,0,0,0,0,0,0,1,0,1,0,230
34542,1.0,0,462,2600,medium,1,1,0,0,0,0,0,0,0,0,1,0,0,1,349
34543,2.0,2,1155,5110,low,1,1,1,0,0,0,0,0,0,0,1,1,0,0,169
34544,1.0,2,231,3400,medium,1,1,0,0,0,0,0,0,0,0,1,0,0,1,401


In [228]:
X['bathrooms'] = X['bathrooms'].apply(lambda x: int(x))
X['rooms'] = X['bathrooms'] + X['bedrooms']
X

,bathrooms,bedrooms,photos,price,TARGET,popular_manager_low,popular_manager_medium,popular_manager_high,manager_not_popular,manager_high,...,manager_low,manager_low_medium,manager_low_high,manager_medium_high,manager_low_medium_high,April,June,May,distance,rooms
0,1,3,231,4495,medium,1,1,0,0,0,...,0,0,0,0,1,0,0,1,419,4
1,1,1,1001,2570,medium,0,0,0,1,0,...,0,0,0,0,1,0,1,0,614,2
2,1,0,462,1795,low,0,0,0,1,0,...,0,0,0,0,1,0,0,1,402,1
3,1,2,308,3400,low,1,1,1,0,0,...,0,0,0,0,1,0,0,1,473,3
4,1,3,231,5695,low,1,0,0,0,0,...,0,0,0,0,1,0,1,0,432,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1,1,308,1950,low,1,1,1,0,0,...,0,0,0,0,1,0,1,0,230,2
34542,1,0,462,2600,medium,1,1,0,0,0,...,0,0,0,0,1,0,0,1,349,1
34543,2,2,1155,5110,low,1,1,1,0,0,...,0,0,0,0,1,1,0,0,169,4
34544,1,2,231,3400,medium,1,1,0,0,0,...,0,0,0,0,1,0,0,1,401,3


In [229]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings("ignore")

Y = X['TARGET']
X = X.drop(columns=['TARGET'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [230]:
logreg = LogisticRegression().fit(X_train, y_train)
logreg_predict = logreg.predict(X_test)

knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)
knn_predict = knn.predict(X_test)

In [232]:
print(f'accuracy on LogReg: {accuracy_score(logreg_predict,y_test)}')
print(f"precision macro on LogReg: {precision_score(logreg_predict,y_test,average='macro')}")
print(f"precision micro on LogReg: {precision_score(logreg_predict,y_test,average='micro')}")
print(f"recall macro on LogReg: {recall_score(logreg_predict,y_test,average='macro')}")
print(f"recall micro on LogReg: {recall_score(logreg_predict,y_test,average='micro')}")
print(f"f1 macro on LogReg: {f1_score(logreg_predict,y_test,average='macro')}")
print(f"f1 micro on LogReg: {f1_score(logreg_predict,y_test,average='micro')}")
print('-------------------------------------------------------------')
print('\n')
print('-------------------------------------------------------------')
print(f'accuracy on KNN: {accuracy_score(knn_predict,y_test)}')
print(f"precision macro on KNN: {precision_score(knn_predict,y_test,average='macro')}")
print(f"precision micro on KNN: {precision_score(knn_predict,y_test,average='micro')}")
print(f"recall macro on KNN: {recall_score(knn_predict,y_test,average='macro')}")
print(f"recall micro on KNN: {recall_score(knn_predict,y_test,average='micro')}")
print(f"f1 macro on KNN: {f1_score(knn_predict,y_test,average='macro')}")
print(f"f1 micro on KNN: {f1_score(knn_predict,y_test,average='micro')}")


accuracy on LogReg: 0.7041968162083936
precision macro on LogReg: 0.3979191270950822
precision micro on LogReg: 0.7041968162083936
recall macro on LogReg: 0.5006688109760352
recall micro on LogReg: 0.7041968162083936
f1 macro on LogReg: 0.40112376612546813
f1 micro on LogReg: 0.7041968162083936
-------------------------------------------------------------


-------------------------------------------------------------
accuracy on KNN: 0.6625180897250362
precision macro on KNN: 0.42075939739965335
precision micro on KNN: 0.6625180897250362
recall macro on KNN: 0.44416183002230697
recall micro on KNN: 0.6625180897250362
f1 macro on KNN: 0.42206220616283324
f1 micro on KNN: 0.6625180897250362


Лучшее значение точности 0.70